# Image Transformation Pipeline

In this notebook we're going to explore the concepts and processes for defining a simple set of operators that will perform image pre-processing before they are feed into our model(s).

In [1]:
!pip3 install kfp --upgrade

Requirement already up-to-date: kfp in /opt/conda/lib/python3.6/site-packages (0.1.27)
You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import kfp.components as comp
import kfp.dsl as dsl
from kfp import compiler

import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
import IPython.display as display
from os import listdir
from os.path import isfile, join

In [ ]:
read_datasets_op = comp.func_to_container_op(read_records)
create_tf_datasets_op = comp.func_to_container_op(create_tf_datasets)
parse_images_op = comp.func_to_container_op(parse_datasets)

In [9]:
@dsl.python_component(
    name="img_preprocessing_pipline",
    description="Image preprocessing pipeline for learning to steer")

def test_nfs(server_ip: str) -> str:
    

def preprocess(dataset_path: str, num_examples: int, output_dir: str,
               batch_size: int) -> str:
    """
    Image preprocessing pipeline
    """
    import json
    import numpy as np
    import tensorflow as tf
    from tensorflow.python.lib.io import file_io
    from sklearn.utils import shuffle
    
    # TODO Move configuration related vars out to config file
    record_dir = 'tfrecords/2148-09-02-2019/tfrecords'
    
    # Create a dictionary describing the features.
    # TODO : define in config file? might be too kludgey from a UX perspective?
    image_feature_description = {
        'timestamp': tf.FixedLenFeature([], tf.float32),
        'image': tf.FixedLenFeature([], tf.string),
        'steering_theta': tf.FixedLenFeature([], tf.float32),
        'accelerator': tf.FixedLenFeature([], tf.float32),
        'height': tf.FixedLenFeature([], tf.int64),
        'width': tf.FixedLenFeature([], tf.int64),
        'capture_height': tf.FixedLenFeature([], tf.int64),
        'capture_width': tf.FixedLenFeature([], tf.int64),
        'capture_fps': tf.FixedLenFeature([], tf.int64),
        'num_channels': tf.FixedLenFeature([], tf.int64)
    }
    
    def read_records(directory: str) -> list:
        """
        Read all files in a directory and return a list containing all 
        of the filenames.
        """
        filenames = [f for f in listdir(directory) if isfile(join(directory, f))]
        return filenames

    def create_tf_datasets(filenames: list) -> list:
        raw_image_datasets = []

        for record in read_records(record_dir):
            raw_image_datasets.append(tf.data.TFRecordDataset(f'{record_dir}/{record}'))

        return raw_image_datasets

    # raw_image_datasets = []

    # for record in tfrecords:
    #     raw_image_datasets.append(tf.data.TFRecordDataset(f'tfrecords/tfrecords/{record}'))

    def parse_image_function(example_proto):
      # Parse the input tf.Example proto using the dictionary above.
        return tf.parse_single_example(example_proto, image_feature_description)

    def parse_datasets(datasets: list) -> list:
        parsed_datasets = []

        for raw_image_dataset in raw_image_datasets:
            parsed_datasets.append(r.map(_parsed_image_function()))

        return parsed_datasets

In [11]:
preprocess_pipeline = preprocess
pipeline_filename = preprocess_pipeline.__name__ = ".image.preprocess.tar.gz"
import kfp.compiler as compiler
compiler.Compiler().compile(preprocess_pipeline, pipeline_filename)

In [14]:

EXPERIMENT_NAME = "LEARN_TO_STEER"
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

run_name = preprocess_pipeline.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename)

Image Transformation Pipeline.ipynb    project.mp4
Lightweight components overview.ipynb  segment-0.avi
mlpipeline-metrics.json		       segment-1.avi
mlpipeline-ui-metadata.json	       segment-2.avi
MNIST_data			       segment-3.avi
projec-1.avi			       tfrecords
projec-1.mp4			       tfrrecords_deserialize.ipynb
projec-2.avi			       Untitled1.ipynb
projec-2.mp4			       Untitled.ipynb
projec-3.avi			       videos
project.avi
